In [2]:
import tkinter as tk
from tkinter import ttk
import os

from DistanceSensor import DistanceSensor
from Speaker import Speaker
from Button import Button
from LED import LED

from Theremin import Theremin
from Note import Note
import DataProcessing
import threading
import queue
import time
from Studio import Studio

#set relevant GPIO pins here
leftBuzzPin = 18
rightBuzzPin = 19
ledPin = 27
buttonPin = 12
noteTrigPin = 24
noteEchoPin = 23
typeTrigPin = 22
typeEchoPin = 21
filename = 'test'

noteDistanceSensor = DistanceSensor(noteTrigPin, noteEchoPin)
typeDistanceSensor = DistanceSensor(typeTrigPin, typeEchoPin)
leftSpeaker = Speaker(leftBuzzPin)
rightSpeaker = Speaker(rightBuzzPin)
button = Button(buttonPin)
light = LED(ledPin)
recording = Recording(filename)
theremin = Theremin(noteDistanceSensor, typeDistanceSensor, leftSpeaker, rightSpeaker)
studio = Studio(theremin, button, light, recording)


LARGEFONT=("Bold Arial", 50)

class tkinterApp(tk.Tk):
     
    # __init__ function for class tkinterApp 
    def __init__(self, *args, **kwargs): 
         
        # __init__ function for class Tk
        tk.Tk.__init__(self, *args, **kwargs)
         
        # creating a container
        container = tk.Frame(self)  
        container.pack(side = "top", fill = "both", expand = True) 
  
        # initializing frames to an empty array
        self.frames = {}  
  
        # iterating through a tuple consisting
        # of the different page layouts
        for F in (StartPage, PlayingPage, SongsPage):
  
            frame = F(container, self)
  
            # initializing frame of that object from
            # startpage, page1, page2 respectively with 
            # for loop
            self.frames[F] = frame 
  
            frame.grid(row = 0, column = 0, sticky ="nsew")
  
        self.show_frame(StartPage)
  
    # to display the current frame passed as
    # parameter
    def show_frame(self, cont):
        frame = self.frames[cont]
        frame.tkraise()
  
# first window frame startpage
  
class StartPage(tk.Frame):
    def __init__(self, parent, controller): 
        tk.Frame.__init__(self, parent)
         
        # label of frame Layout 2
        label = ttk.Label(self, text ="Ultrasonic Theramin", font=LARGEFONT)
         
        # putting the grid in its place by using
        label.grid(row = 0, column = 0, sticky ="nsew")
  
        button1 = ttk.Button(self, text ="Start Playing",
        command = lambda : [controller.show_frame(PlayingPage),  studio.playTheremin()])
     
        # putting the button in its place by
        # using grid
        button1.grid(row = 1, column = 0, padx = 10, pady = 10)
  
        ## button to show frame 2 with text layout2
        button2 = ttk.Button(self, text ="Recorded Songs",
        command = lambda : controller.show_frame(SongsPage))
     
        # putting the button in its place by
        # using grid
        button2.grid(row = 2, column = 0, padx = 10, pady = 10)
  
  
# second window frame page1 
class PlayingPage(tk.Frame):
     
    def __init__(self, parent, controller):
         
        tk.Frame.__init__(self, parent)
        label1 = ttk.Label(self, text ="Ready to Play!", font = LARGEFONT)
        label1.grid(row = 0, column = 4, padx = 10, pady = 10)

        label2 = ttk.Label(self, text ="Move your hand closer to sensor 1 to increase the pitch")
        label2.grid(row = 1, column = 4, padx = 10, pady = 10)

        label3 = ttk.Label(self, text ="Move your hand close to sensor 2 to play sound")
        label3.grid(row = 2, column = 4, padx = 10, pady = 10)

        label4 = ttk.Label(self, text ="Press the button to start recording.")
        label4.grid(row = 3, column = 4, padx = 10, pady = 10)

        label5 = ttk.Label(self, text = "If the light is on, sound is being recorded. Press again to stop.")
        label5.grid(row = 4, column = 4)
  
        # button to show frame 2 with text
        # layout2
        button1 = ttk.Button(self, text ="Stop Playing",
                            command = lambda : [controller.show_frame(StartPage), self.stopPlaying(parent, controller)])
     
        # putting the button in its place 
        # by using grid
        button1.grid(row = 1, column = 1, padx = 10, pady = 10)

    def stopPlaying(self, parent, controller):
        studio.stopTheremin()
        leftSpeaker.stop()
        rightSpeaker.stop()
  
# third window frame page2
class SongsPage(tk.Frame): 
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = ttk.Label(self, text ="Recorded Songs", font = LARGEFONT)
        label.grid(row = 0, column = 1, padx = 10, pady = 10)

        self.selection = os.listdir("./Saves")[0]

        self.var = tk.IntVar()

        self.prevLength = len(os.listdir("./Saves"))

        self.update()

        # button to show frame 2 with text
        # layout2
        button1 = ttk.Button(self, text ="Back to Menu", command = lambda : controller.show_frame(StartPage))
     
        # putting the button in its place by 
        # using grid
        button1.grid(row = 1, column = 0, padx = 10)

        button2 = ttk.Button(self, text = "Update", command = self.update)
        button2.grid(row = 5, column = 0)

        button3 = ttk.Button(self, text = "Rename", command = self.rename)
        button3.grid(row = 2, column = 0)

        button4 = ttk.Button(self, text = "Play", command = lambda : studio.playback(self.selection))
        button4.grid(row = 3, column = 0)

    def sel(self):
        self.selection = self.files[self.var.get()]
        print(self.selection)

    def rename(self):
        top = tk.Toplevel(self)
        ttk.Label(top, text = "Enter new file name (include extension):").grid(row = 0, column = 0)
        entry = ttk.Entry(top)
        entry.grid(row = 0, column = 1)
        button = ttk.Button(top, text = "Save", command = lambda : self.renameFile("./Saves/" + self.selection, "./Saves/" + entry.get(), top))
        button.grid(row = 1, column = 0)

    def renameFile(self, oldName, newName, top):
        os.rename(oldName, newName)
        self.update()
        top.destroy()
        top.update()

    def update(self):
        self.files = os.listdir("./Saves")

        for i in range(self.prevLength):
            ttk.Label(self, text = "                                                   ").grid(row = i + 1, column = 1)
            

        for i in range(len(self.files)):
            ttk.Radiobutton(self, text=self.files[i], variable=self.var, value=i, command=self.sel).grid(row = i + 1, column = 1)

        self.prevLength = len(self.files)

  
# Driver Code
app = tkinterApp()
app.mainloop()

test2
newname.txt
test1.py
